# **Educational Attainment and Migration**

## **Introduction**

This notebook investigates the relationship between **educational attainment** and **migration** among Overseas Filipino Workers (OFWs) from 1995 to 2020. The analysis aims to uncover how different education levels influence the trends of OFW migration over time.

The workflow includes data cleaning, transformation, visualization, and statistical modeling, specifically, **Negative Binomial regression**, to identify key patterns and drivers of migration. 

The findings aim to provide insights into the role of education in shaping migration flows in the Philippines.

## **1. Data Preparation**

### 1.1 Importing Libraries

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
import statsmodels.formula.api as smf

# Setting up inline plotting for Jupyter notebooks
%matplotlib inline 

### 1.2. Loading the Dataset

In [2]:
try:
    edu_df = pd.read_csv('Educational Attainment.csv')
    display(edu_df.head())
except FileNotFoundError:
    print("Error: 'Educational Attainment.csv' not found. Please check the file path.")

,EDUCATIONAL ATTAINMENT,1988,1989,1990,1991,1992,1993,1994,1995,1996,...,2013,2014,2015,2016,2017,2018,2019,2020,TOTAL,%
0,Not of Schooling Age,"5,514","4,792","4,999","4,342","4,729","4,361","4,330","4,081","4,204",...,"5,661","5,842","6,162","5,999","5,239","4,690","4,120",950,"157,221",7.22%
1,No Formal Education,459,"1,254","1,208","1,028",677,670,647,427,453,...,69,95,37,26,116,125,110,17,"10,980",0.50%
2,Elementary Level,"8,847","7,899","8,370","8,070","8,766","9,375","8,727","7,433","8,265",...,"9,129","9,225","11,272","10,943","9,876","8,235","6,821","1,589","272,646",12.52%
3,Elementary Graduate,"3,012","2,721","3,241","3,572","3,144","3,304","3,356","2,579","2,868",...,"2,066","1,980","2,399","2,131","1,861","1,739","1,464",256,"77,441",3.56%
4,High School Level,"7,291","6,967","8,198","8,017","8,650","8,713","8,447","7,546","8,546",...,"8,779","8,665","10,722","10,705","11,060","10,222","8,257","1,949","264,959",12.17%


### 1.3. Initial Data Cleaning

In [3]:
# Drop rows where all values are NaN
edu_df = edu_df.dropna(how='all')
edu_df

,EDUCATIONAL ATTAINMENT,1988,1989,1990,1991,1992,1993,1994,1995,1996,...,2013,2014,2015,2016,2017,2018,2019,2020,TOTAL,%
0,Not of Schooling Age,"5,514","4,792","4,999","4,342","4,729","4,361","4,330","4,081","4,204",...,"5,661","5,842","6,162","5,999","5,239","4,690","4,120",950,"157,221",7.22%
1,No Formal Education,459,"1,254","1,208","1,028",677,670,647,427,453,...,69,95,37,26,116,125,110,17,"10,980",0.50%
2,Elementary Level,"8,847","7,899","8,370","8,070","8,766","9,375","8,727","7,433","8,265",...,"9,129","9,225","11,272","10,943","9,876","8,235","6,821","1,589","272,646",12.52%
3,Elementary Graduate,"3,012","2,721","3,241","3,572","3,144","3,304","3,356","2,579","2,868",...,"2,066","1,980","2,399","2,131","1,861","1,739","1,464",256,"77,441",3.56%
4,High School Level,"7,291","6,967","8,198","8,017","8,650","8,713","8,447","7,546","8,546",...,"8,779","8,665","10,722","10,705","11,060","10,222","8,257","1,949","264,959",12.17%
5,High School Graduate,"5,724","5,625","6,854","7,525","7,627","8,139","8,147","7,302","7,891",...,"8,444","8,422","9,473","8,912","7,893","8,079","7,620","1,900","243,772",11.19%
6,Vocational Level,839,816,"1,115","1,224","1,196","1,230","1,122","1,068","1,054",...,"1,248","1,405","1,587","1,267","1,191","1,075","1,014",234,"34,867",1.60%
7,Vocational Graduate,"1,415","1,433","1,858","2,066","2,662","2,766","2,498","2,132","2,263",...,"4,062","4,468","4,530","4,154","3,614","3,271","2,826",664,"94,207",4.33%
8,College Level,"8,451","8,533","9,848","9,724","10,037","10,064","9,841","8,843","9,996",...,"13,472","13,913","16,699","15,911","12,994","11,089","9,300","2,383","349,565",16.05%
9,College Graduate,"15,614","14,776","16,396","15,835","15,690","16,133","15,810","13,439","13,877",...,"22,841","24,266","27,520","27,162","24,063","23,203","21,875","5,311","608,435",27.94%


In [ ]:
# Drop rows where all values are NaN
edu_df = edu_df.dropna(how='all')

# Drop the last 4 rows as they are still NaN
edu_df = edu_df.drop(edu_df.index[-4:])

# Strip whitespace from the 'EDUCATIONAL ATTAINMENT' column
edu_df['EDUCATIONAL ATTAINMENT'] = edu_df['EDUCATIONAL ATTAINMENT'].str.strip()

# Remove rows that are not meaningful for analysis
edu_df = edu_df[~edu_df['EDUCATIONAL ATTAINMENT'].isin(['TOTAL', 'Not Reported / No Response'])]

# Reset the index after removing rows
edu_df = edu_df.reset_index(drop=True)

# Show info and summary
edu_df.info()
display(edu_df.describe(include='all'))

### 1.4. Data Transformation: Wide to Long Format

In [ ]:
# Identify year columns (all columns except 'EDUCATIONAL ATTAINMENT', 'TOTAL', and '%')
year_columns = [col for col in edu_df.columns if col.isdigit()]

# Remove commas and convert year columns to numeric
for col in year_columns:
    edu_df[col] = pd.to_numeric(edu_df[col].astype(str).str.replace(',', ''), errors='coerce')

edu_df['TOTAL'] = pd.to_numeric(edu_df['TOTAL'].astype(str).str.replace(',', ''), errors='coerce')
edu_df['%'] = pd.to_numeric(edu_df['%'].astype(str).str.replace('%', ''), errors='coerce')

# Reshape to long format
edu_long = edu_df.melt(id_vars=['EDUCATIONAL ATTAINMENT'], 
                       value_vars=year_columns, 
                       var_name='YEAR', 
                       value_name='COUNT')
edu_long['YEAR'] = pd.to_numeric(edu_long['YEAR'], errors='coerce')
edu_long = edu_long[edu_long['YEAR'] >= 1995].reset_index(drop=True)

display(edu_long.head())


Converting the dataset to long format is essential for analysis because it organizes the data into a structure where _each row_ represents a _single observation_ (e.g., a specific education level in a specific year), which is required for **statistical modeling**, such as regression analysis. It also simplifies operations like grouping, filtering, and aggregating data, making it easier to analyze trends over time or compare categories.

Additionally, long format ensures compatibility with Python libraries like _statsmodels_, _seaborn_, and _matplotlib_, which are designed to work with tidy data.

### 1.5. Mapping Educational Attainment to Broader Groups

In [ ]:
# Define the mapping of educational attainment to broader groups
education_mapping = {
    'Not of Schooling Age': 'No Schooling',
    'No Formal Education': 'No Schooling',
    'Elementary Level': 'Basic Education',
    'Elementary Graduate': 'Basic Education',
    'High School Level': 'Secondary Education',
    'High School Graduate': 'Secondary Education',
    'Vocational Level': 'Vocational/Tech',
    'Vocational Graduate': 'Vocational/Tech',
    'College Level': 'Tertiary (College)',
    'College Graduate': 'Tertiary (College)',
    'Post Graduate Level': 'Advanced (Postgrad)',
    'Post Graduate': 'Advanced (Postgrad)',
    'Non-Formal Education': 'No Schooling',
}

# Map the detailed categories to broader groups
edu_long['EDUCATION'] = edu_long['EDUCATIONAL ATTAINMENT'].map(education_mapping)

# After mapping education
edu_long = edu_long.dropna(subset=['EDUCATION'])

edu_long

Defining a mapping of educational attainment to broader groups helps simplify and standardize the dataset, which makes it easier to analyze and interpret. By grouping similar categories (e.g., "Elementary Level" and "Elementary Graduate" into "Basic Education"), the dataset becomes _more manageable_ and _reduces redundancy_. This also ensures that the analysis focuses on meaningful distinctions between education levels rather than being overwhelmed by overly granular categories.

### 1.6. Aggregating by Education Group and Year

In [ ]:
# Aggregate the data by education group and year
grouped_edu_long = edu_long.groupby(['EDUCATION', 'YEAR'])['COUNT'].sum().reset_index()

grouped_edu_long

This step is crucial for eliminating duplicate or redundant rows and consolidating the data into a format suitable for analysis. By grouping the data, it becomes easier to identify trends, compare migration patterns across education levels, and perform statistical modeling, as the dataset is now structured with unique combinations of education groups and years.

### 1.7. Setting Education as a Categorical Variable

In [ ]:
# Set the desired reference category (e.g., 'No Schooling')
grouped_edu_long['EDUCATION'] = pd.Categorical(
    grouped_edu_long['EDUCATION'],
    categories=[
        'No Schooling', 
        'Basic Education', 
        'Secondary Education', 
        'Vocational/Tech', 
        'Tertiary (College)', 
        'Advanced (Postgrad)'
    ],
    ordered=True
)
grouped_edu_long

This code sets the _EDUCATION_ column as a **categorical** variable with a specific order of categories. It ensures that the education levels are treated as an ordered categorical variable, which is crucial for regression analysis. The order specifies the hierarchy of education levels, with "No Schooling" as the reference category (the baseline for comparison in the regression model).Moreover, this allows the statistical model to interpret the effects of other education levels **relative** to the reference category.

## **2. Data Visualization**

In [ ]:
# Define a consistent colormap for both graphs
education_colors = {
    'No Schooling': '#1f77b4',  # Blue
    'Basic Education': '#ff7f0e',  # Orange
    'Secondary Education': '#2ca02c',  # Green
    'Vocational/Tech': '#d62728',  # Red
    'Tertiary (College)': '#9467bd',  # Purple
    'Advanced (Postgrad)': '#8c564b',  # Brown
}

# Ensure the colors are applied in the correct order
education_order = list(education_colors.keys())


### 2.1. Line Plot: Emigrants by Education Group Over Time

In [ ]:
plt.figure(figsize=(12, 8))
for edu in education_order:
    plt.plot(grouped_edu_long[grouped_edu_long['EDUCATION'] == edu]['YEAR'],
             grouped_edu_long[grouped_edu_long['EDUCATION'] == edu]['COUNT'],
             label=edu, color=education_colors[edu])
plt.title('Trends in Emigrants by Education Group Over Time')
plt.xlabel('Year')
plt.ylabel('Number of Emigrants')
plt.legend(title='Education Group', bbox_to_anchor=(1.05, 1), loc='upper left')
plt.grid(True)
plt.tight_layout()
plt.show()

**INTERPRETATION**

The graph shows the **trends in the number of emigrants over time (1995–2020)**, categorized by education group. The y-axis represents the number of emigrants, while the x-axis represents the years. Each line corresponds to a specific education group, with the legend indicating the group names.

From the graph, it is evident that individuals with **Tertiary (College)** education consistently have the highest number of emigrants, peaking around 2015 before declining sharply by 2020. This suggests that tertiary-educated individuals are the most mobile or in demand internationally. In contrast, groups like **Advanced (Postgrad)** and **Vocational/Tech** have significantly lower emigration counts.

The relatively stable trends for **Basic Education** and **Secondary Education** indicate less variation in migration patterns for these groups. Lastly, there are notable declines across most groups after 2015.

### 2.2. Stacked Area Chart: Proportion of Emigrants by Education Group

In [ ]:
# Create a pivot table for the stacked area chart
pivot_data = grouped_edu_long.pivot(index='YEAR', columns='EDUCATION', values='COUNT')

# Stacked Area Chart
plt.figure(figsize=(12, 8))
pivot_data.plot(kind='area', stacked=True, color=[education_colors[edu] for edu in education_order], alpha=0.8)
plt.title('Proportion of Emigrants by Education Group Over Time')
plt.xlabel('Year')
plt.ylabel('Number of Emigrants')
plt.legend(title='Education Group', bbox_to_anchor=(1.05, 1), loc='upper left')
plt.grid(True)
plt.show()

**INTERPRETATION**

This stacked area chart is similar to the previous line chart in that it shows the **trends in emigrants by education group over time (1995–2020)**. However, it provides additional insights by visualizing the **proportions of each education group** relative to the total number of emigrants in each year. The stacked format highlights how the contributions of different education groups collectively make up the total emigration numbers.

Unlike the line chart, this chart emphasizes how each education group contributes to the overall emigration numbers. For example, **Tertiary (College)** and **Secondary Education** dominate the total emigration, but the stacked format displays further how their proportions change over time.

## **3. Statistical Modeling**

### 3.1. Model Selection: Poisson vs. Negative Binomial

Given the dataset, the best regression analysis to investigate how education levels in the Philippines impact the migration of Overseas Filipino Workers (OFWs) is either **Poisson Regression** or **Negative Binomial Regression**. This is because the dependent variable (`COUNT`) represents count data (the number of OFWs), which is non-negative and discrete.

Poisson Regression is specifically designed to model count data and assumes that the mean and variance of the dependent variable are equal. It allows us to estimate the effect of categorical predictors like `EDUCATION` on the number of OFWs while accounting for the temporal dimension (`YEAR`).

However, if overdispersion is detected (i.e., the variance of `COUNT` is significantly larger than the mean), **Negative Binomial Regression** would be a better alternative, as it introduces a dispersion parameter to handle the extra variability. Both models can provide insights into how different education levels influence migration patterns, with the coefficients indicating the relative impact of each education group compared to a reference category (e.g., "No Schooling").

### 3.2. Checking for Overdispersion

In [ ]:
# Calculate mean and variance of COUNT
mean_count = grouped_edu_long['COUNT'].mean()
variance_count = grouped_edu_long['COUNT'].var()

print(f"Mean of COUNT: {mean_count}")
print(f"Variance of COUNT: {variance_count}")

**INTERPRETATION**

The results show that the **mean of `COUNT`** is `11164.141025641025`, while the **variance of `COUNT`** is ` 109253782.56062864`. The variance is significantly larger than the mean, indicating **overdispersion** in the data. This violates the assumption of Poisson Regression, which assumes that the mean and variance of the dependent variable are equal.


Given the presence of overdispersion, **Negative Binomial Regression** is the most appropriate model for this analysis. Negative Binomial Regression introduces a dispersion parameter (`alpha`) to account for the extra variability in the data, making it _more robust_ than Poisson Regression when the variance exceeds the mean. This model will allow us to estimate the impact of education levels (`EDUCATION`) on the migration of OFWs (`COUNT`) while properly handling the overdispersion in the data.

### 3.3. Negative Binomial Regression (Education Only)

In [ ]:
# Define the dependent variable (COUNT)
Y = grouped_edu_long['COUNT']

# Simplify the model by removing YEAR
X = pd.get_dummies(grouped_edu_long[['EDUCATION']], drop_first=True)
X = sm.add_constant(X)

# Convert boolean columns to numeric (int) to avoid dtype issues
X = X.astype(int)

# Fit the Negative Binomial model
neg_bin_model = sm.NegativeBinomial(Y, X).fit()

# Print the regression summary
print(neg_bin_model.summary())

**INTERPRETATION**

The Negative Binomial Regression model results indicate that education levels significantly impact the migration of Overseas Filipino Workers (OFWs). The **reference category** is "_No Schooling_," and the coefficients represent the log-transformed effect of each education group relative to this baseline. Significant findings include:

1. **Positive Effects**:
   - **Basic Education**: Migration counts are **104% higher** compared to "No Schooling" (`exp(0.7142) ≈ 2.04`).
   - **Secondary Education**: Migration counts are **206% higher** compared to "No Schooling" (`exp(1.1229) ≈ 3.06`).
   - **Tertiary (College)**: Migration counts are **503% higher** compared to "No Schooling" (`exp(1.7853) ≈ 6.10`).

2. **Negative Effects**:
   - **Advanced (Postgrad)**: Migration counts are **60% lower** compared to "No Schooling" (`exp(-0.9246) ≈ 0.40`).
   - **Vocational/Tech**: Migration counts are **18% lower** compared to "No Schooling" (`exp(-0.2043) ≈ 0.82`).

The **dispersion parameter (`alpha = 0.1213`)** confirms that the Negative Binomial model is appropriate, as it accounts for overdispersion in the data. The **Pseudo R-squared value (0.1009)** indicates that the model explains ~10.1% of the variation in migration counts.

### 3.4. Negative Binomial Regression with Temporal Effects

Incorporating temporal effects into the regression model is essential because migration patterns are influenced not only by educational attainment but also by _year-specific factors_ such as economic conditions, policy changes, and global events. By including **_year_** as a _categorical variable_, the model controls for these time-varying influences, thereby reducing omitted variable bias and improving the accuracy of the estimated effects of education.

Moreover, this approach ensures that the observed relationship between education and migration is not confounded by unobserved shocks or trends unique to particular years, resulting in more robust and interpretable findings. The code below correctly implements this strategy by generating year dummies and including them in the regression, which aligns with best practices for longitudinal data analysis.

In [ ]:
# Define the dependent variable (COUNT)
Y = grouped_edu_long['COUNT']

# Convert YEAR to a categorical variable to include it in the model
grouped_edu_long['YEAR_cat'] = grouped_edu_long['YEAR'].astype('category')

# Create dummy variables for EDUCATION and YEAR (categorical)
X = pd.get_dummies(grouped_edu_long[['YEAR_cat', 'EDUCATION']], drop_first=True)
X = sm.add_constant(X)

# Convert boolean columns to numeric (int) to avoid dtype issues
X = X.astype(int)

After accounting for temporal effects to control for year-specific influences, it is equally important to assess the relationships among the predictor variables themselves. Checking for multicollinearity before fitting the Negative Binomial model ensures that the independent variables included in the analysis are not excessively correlated, which could otherwise distort coefficient estimates and inflate standard errors. By calculating the Variance Inflation Factor (VIF) for each predictor, the analysis can identify and address any problematic multicollinearity, thereby supporting the validity and interpretability of the model’s results. This diagnostic step, as implemented in the code, confirms that the predictors are suitable for reliable regression analysis.

### 3.5. Multicollinearity Check (VIF)

In [ ]:
# Check for multicollinearity using Variance Inflation Factor (VIF)
from statsmodels.stats.outliers_influence import variance_inflation_factor

# Exclude the constant column for VIF calculation
X_no_const = X.loc[:, X.columns != 'const']

vif_data = pd.DataFrame()
vif_data["feature"] = X_no_const.columns
vif_data["VIF"] = [variance_inflation_factor(X_no_const.values, i) for i in range(X_no_const.shape[1])]
print(vif_data)

**INTERPRETATION**

The Variance Inflation Factor (VIF) results indicate that all predictor variables in the model exhibit low levels of multicollinearity. Specifically, the VIF values for the year dummy variables are consistently 1.16, while the VIF values for the education group dummies are 1.81. Since all VIF values are well below the commonly used threshold of 5 (and even the more conservative threshold of 2), there is no evidence of problematic multicollinearity among the predictors. This finding is significant because it confirms that the estimated effects of each education group and year are not distorted by excessive correlation with other variables, thereby supporting the reliability and interpretability of the regression coefficients. Consequently, the model is appropriately specified for subsequent analysis, and no further action is required to address multicollinearity.

### 3.6. Model Fit and Coefficient Interpretation

Given that the VIF results confirm the absence of problematic multicollinearity among the predictors, the dataset is now appropriately specified for regression analysis. With this diagnostic step completed, it is now appropriate to proceed with fitting the Negative Binomial model to estimate the effects of education and temporal factors on migration counts.

In [ ]:
# Fit Negative Binomial model with education and year
print("Fitting Negative Binomial Regression Model...")
neg_bin_model = sm.NegativeBinomial(Y, X)
neg_bin_results = neg_bin_model.fit()
print(neg_bin_results.summary())

**INTERPRETATION**

The results of the Negative Binomial Regression model indicate that both education levels and specific years significantly impact the migration of Overseas Filipino Workers (OFWs). The **reference category** for `YEAR` is _1995_, and for `EDUCATION` is _No Schooling_. The coefficients represent the log-transformed effect of each category relative to these baselines.

The **Pseudo R-squared (0.2144)** indicates that the model explains ~21.4% of the variation in migration counts, which is a significant improvement compared to models without temporal effects.

- **Model Fit:**
  - Log-Likelihood: **-1262.7** (higher is better)
  - LLR p-value: **< 0.001** (model is statistically significant)
  - Alpha: **0.0118, p < 0.001** (overdispersion present; Negative Binomial is appropriate)

- **Year Effects (relative to 1995):**
  - **Significant positive coefficients** for 2004–2018 (e.g., 2006: 0.4487, p < 0.001; 2010: 0.4729, p < 0.001)  
    → Migration counts were higher in these years compared to 1995.
  - **2020:** Coefficient = **-1.3472, p < 0.001**  
    → Migration dropped sharply (about 74% lower than 1995, likely due to COVID-19).
  - **1998, 1999:** Significant negative coefficients (p < 0.001)  
    → Migration counts were lower than 1995.

- **Education Effects (relative to "No Schooling"):**
  - **Basic Education:** Coefficient = **0.7209, p < 0.001**  
    → About **106% higher** migration counts (`exp(0.7209) ≈ 2.06`).
  - **Secondary Education:** Coefficient = **1.1456, p < 0.001**  
    → About **214% higher** migration counts (`exp(1.1456) ≈ 3.14`).
  - **Tertiary (College):** Coefficient = **1.7855, p < 0.001**  
    → About **496% higher** migration counts (`exp(1.7855) ≈ 5.96`).
  - **Vocational/Tech:** Coefficient = **-0.2124, p < 0.001**  
    → About **19% lower** migration counts (`exp(-0.2124) ≈ 0.81`).
  - **Advanced (Postgrad):** Coefficient = **-0.9405, p < 0.001**  
    → About **61% lower** migration counts (`exp(-0.9405) ≈ 0.39`).

- **Summary of Significant Findings:**
  - Migration is significantly higher for those with higher education, especially college.
  - Migration dropped dramatically in 2020.
  - Vocational/Tech and Advanced (Postgrad) groups migrate less than those with no schooling.
  - Most years after 2004 saw higher migration than 1995.

## **4. Model Diagnostics**

After fitting the Negative Binomial model and reviewing the regression results, the next appropriate step is to **check the model’s residuals for patterns or violations of assumptions**. Typically, this involves:

1. **Plotting residuals vs. fitted values** to visually inspect for non-random patterns, heteroskedasticity, or outliers.
2. **Conducting the Durbin-Watson test** to formally check for autocorrelation in the residuals, especially since your data is time series (panel) in nature.

Both steps are standard and complementary. However, **plotting residuals vs. fitted values** is usually done first as it provides a quick visual diagnostic, followed by the Durbin-Watson test for a more formal assessment of autocorrelation.

### 4.1. Residuals vs. Fitted Values Plot

In [ ]:
# Plot residuals vs. fitted values
import matplotlib.pyplot as plt

fitted_vals = neg_bin_results.fittedvalues
residuals = neg_bin_results.resid

plt.figure(figsize=(8, 5))
plt.scatter(fitted_vals, residuals, alpha=0.7)
plt.axhline(0, color='red', linestyle='--')
plt.xlabel('Fitted Values')
plt.ylabel('Residuals')
plt.title('Residuals vs. Fitted Values')
plt.grid(True)
plt.show()

**INTERPRETATION**

The plot shows the residuals (differences between observed and predicted values) on the y-axis and the fitted (predicted) values on the x-axis. Ideally, residuals should be randomly scattered around zero, with **no clear pattern** or trend. This would indicate that the model’s assumptions are reasonably met.

There appears to be some spread in the residuals as fitted values increase, suggesting possible **heteroskedasticity** (variance of residuals increases with fitted values). Moreover, there are a few points with large positive or negative residuals, which may be **outliers**. However, there is no strong non-random pattern (such as a curve or systematic trend) is immediately visible, but the increasing spread is notable.

### 4.2. Durbin-Watson Test for Autocorrelation

The Durbin-Watson test should be conducted after plotting residuals vs. fitted values to formally assess the presence of autocorrelation in the model’s residuals. While the residual plot provides a visual check for non-random patterns, it cannot reliably detect serial correlation, especially in time series or panel data. Autocorrelation in residuals violates the assumption of independent errors, which can lead to underestimated standard errors and inflated significance of predictors. By performing the Durbin-Watson test, the analysis objectively determines whether residuals are independent, thereby ensuring the validity of statistical inference and supporting the robustness of the model’s conclusions.

In [ ]:
from statsmodels.stats.stattools import durbin_watson

# Get residuals from your fitted model
residuals = neg_bin_results.resid

# Durbin-Watson test
dw_stat = durbin_watson(residuals)
print(f"Durbin-Watson statistic: {dw_stat}")

**INTERPRETATION**

The Durbin-Watson statistic ranges from 0 to 4, where a value near 2 indicates no autocorrelation, values below 2 indicate positive autocorrelation, and values above 2 indicate negative autocorrelation. However, for this specific test, the result is approximately **0.311**.


A value of **0.31** is much lower than 2, indicating **strong positive autocorrelation** in the residuals, which violates the assumption of independent errors. Thus, this can lead to underestimated standard errors and inflated significance of predictors, making inference from the model unreliable.

To address this, we must use robust standard errors (such as HAC) to obtain valid statistical inference and correct for autocorrelation in the model.

### 4.3. Negative Binomial Regression with Robust (HAC) Standard Errors

In [ ]:
# Fit the Negative Binomial model with robust (HAC) standard errors
neg_bin_model = sm.NegativeBinomial(Y, X) # Specify the model
neg_bin_results = neg_bin_model.fit(cov_type='HAC', cov_kwds={'maxlags': 1}) # Fit the model with HAC standard errors

print(neg_bin_results.summary())

**INTERPRETATION**

After fitting the Negative Binomial model with robust (HAC) standard errors, we established that education level and year are significant predictors of migration, and that our results are statistically robust even in the presence of autocorrelation. However, as with any rigorous analysis, it is essential to **visually inspect the residuals of the robust model** to ensure that no important model assumptions are violated and that our conclusions remain valid.

As a final diagnostic, we plot the residuals against the fitted values for the robust model. This step allows us to visually confirm that the model fits well and that no systematic patterns, heteroskedasticity, or outliers remain unaddressed. If the residuals appear randomly scattered around zero, it reinforces the integrity of our results and the reliability of our conclusions.



### 4.4. Residuals vs. Fitted Values (Robust Model)

In [ ]:
# Plot residuals vs. fitted values for the robust model
fitted_vals = neg_bin_results.fittedvalues
residuals = neg_bin_results.resid

plt.figure(figsize=(8, 5))
plt.scatter(fitted_vals, residuals, alpha=0.7)
plt.axhline(0, color='red', linestyle='--')
plt.xlabel('Fitted Values')
plt.ylabel('Residuals')
plt.title('Residuals vs. Fitted Values (Robust Model)')
plt.grid(True)
plt.show()

**INTERPRETATION**

The plot displays the residuals (differences between observed and predicted values) on the y-axis and the fitted values on the x-axis for the robust Negative Binomial model. **Ideally**, residuals should be randomly scattered around zero, with no clear pattern or trend. This would indicate that the model’s assumptions are reasonably met.


Based on the following plot, the residuals are generally centered around zero, but as fitted values increase, the spread of residuals also increases. This again suggests the presence of **heteroskedasticity** where the variance of residuals grows with larger fitted values.


Moreover, there are some points with large positive or negative residuals, indicating possible **outliers** or influential observations, similar to the results of the previous plot. However, no strong systematic pattern (such as a curve or funnel shape) is visible, but the increasing spread is notable.


Since the robust (HAC) standard errors already account for heteroskedasticity and autocorrelation, the model’s statistical inference remains valid. However, the presence of heteroskedasticity and outliers suggests that while the model is appropriate, further **refinement** (such as investigating influential points or alternative model specifications) could be considered for even greater robustness.

### 4.5. Durbin-Watson Test (Robust Model)

In [ ]:
from statsmodels.stats.stattools import durbin_watson

residuals = neg_bin_results.resid
dw_stat = durbin_watson(residuals)
print(f"Durbin-Watson statistic: {dw_stat}")

**INTERPRETATION**

The Durbin-Watson statistic for the residuals of the robust Negative Binomial model is approximately **0.31**, which is substantially lower than the ideal value of 2. This result indicates that **strong positive autocorrelation remains present in the residuals**, even after accounting for heteroskedasticity and autocorrelation using robust (HAC) standard errors. While the use of robust errors ensures that statistical inference remains valid, the persistence of autocorrelation suggests that some temporal patterns or dependencies in the data are not fully captured by the current model specification. This limitation should be considered when interpreting the results, as it may affect the precision of the estimated effects.

With these considerations in mind, the following section summarizes the key findings, limitations, and recommendations based on the analysis.

## **5. Summary of Analysis**

This study investigated the relationship between educational attainment and migration among Overseas Filipino Workers (OFWs) using data from 1995 to 2020. The dataset was cleaned, standardized, and reshaped into a long format suitable for statistical analysis. Educational attainment categories were mapped into broader groups, and the data was aggregated by education group and year. Visualizations (line and stacked area charts) illustrated migration trends across education levels over time.

For the main analysis, a Negative Binomial regression model was employed due to the presence of overdispersion in the count data. The model included both education group and year as predictors, allowing for the estimation of their effects on migration counts. Robust (HAC) standard errors were used to account for autocorrelation and heteroskedasticity in the residuals, ensuring valid statistical inference.

## **6. Key Findings**

- **Education Level:**  
  Higher educational attainment is strongly associated with increased migration. OFWs with tertiary (college) education have the highest migration counts, while those with vocational/technical or advanced (postgraduate) education migrate less than those with no schooling.
- **Temporal Effects:**  
  Migration counts vary significantly by year. There was a notable increase in migration from 2004 to 2018, and a sharp decline in 2020, likely due to the COVID-19 pandemic.
- **Model Fit:**  
  The Negative Binomial model with education and year predictors explains about 21% of the variation in migration counts, indicating a moderate fit.


## **7. Limitations**

- **Residual Autocorrelation:**  
  Despite using robust (HAC) standard errors, the Durbin-Watson statistic indicated strong positive autocorrelation in the residuals. This suggests that some temporal dependencies remain unmodeled, which could affect the precision of coefficient estimates.
- **Heteroskedasticity and Outliers:**  
  Residual plots revealed increasing variance with higher fitted values and the presence of outliers. While robust errors mitigate these issues for inference, they may still impact model fit.
- **Omitted Variables:**  
  The model does not account for other potential factors influencing migration, such as economic indicators, policy changes, or international demand for labor.
- **Data Limitations:**  
  The analysis is limited to available data and aggregated categories, which may mask more nuanced patterns within subgroups.


## **8. Recommendations for Future Research**

- **Modeling Temporal Dynamics:**  
  Future studies should consider advanced time-series or panel data models (e.g., models with lagged dependent variables, ARIMA, or mixed-effects models) to better capture autocorrelation and temporal dependencies.
- **Inclusion of Additional Predictors:**  
  Incorporate economic, social, and policy variables to provide a more comprehensive understanding of migration drivers.
- **Granular Data:**  
  Use more granular or disaggregated data (e.g., by region, occupation, or gender) to uncover subgroup-specific trends.
- **Addressing Outliers:**  
  Investigate and address influential outliers or leverage robust regression techniques to minimize their impact.
- **Longitudinal Tracking:**  
  If possible, track individuals over time to distinguish between repeated and first-time migrants.

## **9. Conclusion**

This analysis demonstrates that educational attainment and year are significant predictors of OFW migration, with higher education linked to greater migration likelihood. While robust statistical methods were used, some limitations remain due to residual autocorrelation and unmodeled factors. Future research can build on this foundation by employing more sophisticated models and richer datasets to further elucidate the dynamics of migration among Filipino workers.